In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import glob
import os
import warnings
import numpy as np

from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

from sound_utils import extract_signal_features, generate_dataset, load_sound_file, extract_log_mel_windows_VAE, generate_dataset_from_list_VAE
from misc import build_files_list, dump_pickle, load_pickle
from autoencoder import (vae,encoder_model,decoder_model)
from eval_perf import (
    get_prediction,
    plot_confusion_matrix,
    plot_histogram_by_class,
    plot_loss_per_epoch,
    plot_pr_curve,
    plot_roc_curve,
)

np.random.seed(42)

In [ ]:
import tensorflow as tf

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.python.client import device_lib

tf.random.set_seed(42)

In [ ]:
from bokeh.io import export_svgs, output_notebook, reset_output
from bokeh.models import BoxAnnotation, ColumnDataSource, HoverTool
from bokeh.plotting import figure, show
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
    average_precision_score,
    precision_recall_curve,
    roc_auc_score,
    roc_curve,
)

output_notebook()

#Processing pipeline


1.   Load data
2.   Split into training, test and validation sets
3.   Extract log-Mel spectrograms
4.   Save the spectrograms



In [ ]:
root_dir = "/.../ToyCar_data"
DATA_PATH = "/.../ToyCar_data/ToyADMOS-anomaly-detection"
MODEL_PATH = "/.../ToyADMOS-anomaly-detection"

In [ ]:
# Load full file lists (assuming build_files_list returns two lists)
normal_files, abnormal_files = build_files_list(root_dir)

# Randomly sample 50% of each
normal_sample_indices = np.random.choice(len(normal_files), size=len(normal_files) // 2, replace=False)
abnormal_sample_indices = np.random.choice(len(abnormal_files), size=len(abnormal_files) // 2, replace=False)

normal_files_sampled = [normal_files[i] for i in normal_sample_indices]
abnormal_files_sampled = [abnormal_files[i] for i in abnormal_sample_indices]

# Create labels for the sampled files
normal_labels = np.zeros(len(normal_files_sampled))
abnormal_labels = np.ones(len(abnormal_files_sampled))

# Split normal files into train/test
train_files, test_files, train_labels, test_labels = train_test_split(
    normal_files_sampled, normal_labels, train_size=0.8, random_state=42, shuffle=True
)

# Add abnormal files to test set
test_files = np.concatenate((test_files, abnormal_files_sampled), axis=0)
test_labels = np.concatenate((test_labels, abnormal_labels), axis=0)

# Shuffle test set
test_indices = np.arange(len(test_files))
np.random.shuffle(test_indices)

test_files = test_files[test_indices]
test_labels = test_labels[test_indices]

# Print dataset stats
print(
    f"Train set has {train_labels.shape[0]} signals including abnormal {train_labels.sum():.0f} signals, "
    f"but test set has {test_labels.shape[0]} signals including abnormal {test_labels.sum():.0f} signals."
)

Train set has 2160 signals including abnormal 0 signals, but test set has 1069 signals including abnormal 529 signals.


In [ ]:
# Extract spectrograms for training set
n_fft = 1024
hop_length = 512
n_mels = 80
frames = 5

train_data_path = os.path.join(DATA_PATH, "dataset", "train_data_VAE" + ".pkl")

if os.path.exists(train_data_path):
    print("Train data already exists, loading from file...")
    train_data = load_pickle(train_data_path)

else:
    train_data = generate_dataset_from_list_VAE(train_files)
    print("Saving train data to disk...")
    dump_pickle(train_data_path, train_data)
    print("Done.")

print(f"Train data has a {train_data.shape} shape.") # (num_windows_total, 5, 128, 1)

Extracting features: 100%|██████████| 2160/2160 [32:01<00:00,  1.12it/s]


Saving train data to disk...
Done.
Train data has a (734400, 5, 80, 1) shape.


#VAE

In [ ]:
# Get the encoder, decoder and 'master' model (called vae) #input_shape=(256,626,1)
encoder, decoder, vae = get_models(input_shape=(5,80,1), latent_dim=LATENT_DIM)

vae.summary()

# Define the loss functions and optimizers
optimizer = tf.keras.optimizers.Adam()
loss_metric = tf.keras.metrics.Mean()
mse_loss = tf.keras.losses.MeanSquaredError()

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 5, 80, 1)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_9 (Functional)       │ [(None, 2), ()]        │       101,028 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_10 (Functional)      │ (None, 5, 80, 20)      │       106,300 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 207,328 (809.88 KB)

 Trainable params: 206,736 (807.56 KB)

 Non-trainable params: 592 (2.31 KB)

In [ ]:
encoder.summary()
decoder.summary()

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_input       │ (None, 5, 80, 1)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encode_conv1        │ (None, 5, 40, 20) │        200 │ encoder_input[0]… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5, 40, 20) │         80 │ encode_conv1[0][… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encode_conv2        │ (None, 5, 20, 40) │      7,240 │ batch_normalizat… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5, 20, 40) │        160 │ encode_conv2[0][… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encode_conv3        │ (None, 5, 10, 80) │     28,880 │ batch_normalizat… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5, 10, 80) │        320 │ encode_conv3[0][… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encode_flatten      │ (None, 4000)      │          0 │ batch_normalizat… │
│ (Flatten)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ encode_dense        │ (None, 16)        │     64,016 │ encode_flatten[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 16)        │         64 │ encode_dense[0][… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ latent_mu (Dense)   │ (None, 2)         │         34 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ latent_sigma        │ (None, 2)         │         34 │ batch_normalizat… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sampling_3          │ (None, 2)         │          0 │ latent_mu[0][0],  │
│ (Sampling)          │                   │            │ latent_sigma[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ kld_layer_3         │ ()                │          0 │ latent_mu[0][0],  │
│ (KLDLayer)          │                   │            │ latent_sigma[0][… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 101,028 (394.64 KB)

 Trainable params: 100,716 (393.42 KB)

 Non-trainable params: 312 (1.22 KB)

Model: "functional_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 2)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decode_dense1 (Dense)           │ (None, 4000)           │        12,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decode_reshape (Reshape)        │ (None, 5, 10, 80)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decode_conv2d_1                 │ (None, 5, 20, 80)      │        57,680 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_27          │ (None, 5, 20, 80)      │           320 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decode_conv2d_2                 │ (None, 5, 40, 40)      │        28,840 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_28          │ (None, 5, 40, 40)      │           160 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decode_conv2d_3                 │ (None, 5, 80, 20)      │         7,220 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_29          │ (None, 5, 80, 20)      │            80 │
│ (BatchNormalization)            │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 106,300 (415.23 KB)

 Trainable params: 106,020 (414.14 KB)

 Non-trainable params: 280 (1.09 KB)

In [ ]:
# Define global constants to be used in this notebook
%%time

vae.compile(optimizer='adam',
            loss='mean_squared_error')

history = vae.fit(train_data,train_data,
                  batch_size=512,
                  epochs=100,
                  callbacks=[EarlyStopping(monitor="val_loss", patience=10)],
                  validation_split=0.2,
                  verbose=1,
                  shuffle=True)

Epoch 1/100
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 31s 16ms/step - loss: 1620.1063 - val_loss: 1524.3850
Epoch 2/100
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 1491.1936 - val_loss: 1407.0347
Epoch 3/100
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - loss: 1377.8563 - val_loss: nan
Epoch 4/100
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - loss: 1268.3795 - val_loss: 1188.9244
Epoch 5/100
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - loss: 1163.3588 - val_loss: 1083.4528
Epoch 6/100
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - loss: 1064.4587 - val_loss: 986.1252
Epoch 7/100
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - loss: 970.7115 - val_loss: 895.9705
Epoch 8/100
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 881.9967 - val_loss: 817.2611
Epoch 9/100
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 798.2635 - val_loss: 738.5738
Epoch 10/100
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - loss: 719.5186 - val_loss: 670.3243
Epoch 11/100
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 11s 

In [ ]:
MODEL_NAME="Model2_VAE"

#Performance Evaluation

In [ ]:
plot_loss_per_epoch(
    history, model_name=MODEL_NAME
)

In [ ]:
from tqdm import tqdm
import numpy as np

recon_errors = []

for file_path in tqdm(test_files, desc="Evaluating test files"):
    # Extract log-mel spectrogram windows
    features = extract_log_mel_windows_VAE(
        file_path,
        sr=16000,
        n_fft=1024,
        hop_length=512,
        n_mels=80,
        frames=5
    )

    if features.size == 0:
        recon_errors.append(np.nan)
        continue

    # Predict reconstructed features from the model
    predictions = vae.predict(features, verbose=0)

    # Compute mean squared error per window and average over all windows
    mse_per_window = np.mean(np.square(features - predictions), axis=(1, 2, 3))  # shape: (num_windows,)
    file_error = np.mean(mse_per_window)
    recon_errors.append(file_error)

Evaluating test files: 100%|██████████| 1069/1069 [17:58<00:00,  1.01s/it]


In [ ]:
recon_errors

[np.float32(9.397431),
 np.float32(8.412422),
 np.float32(11.76497),
 np.float32(8.493022),
 np.float32(18.887922),
 np.float32(15.891406),
 np.float32(18.301529),
 np.float32(15.556434),
 np.float32(9.193243),
 np.float32(16.036133),
 np.float32(7.980308),
 np.float32(16.531422),
 np.float32(10.295995),
 np.float32(9.448217),
 np.float32(8.307544),
 np.float32(8.077961),
 np.float32(9.211384),
 np.float32(14.761918),
 np.float32(8.962851),
 np.float32(15.5492735),
 np.float32(9.279444),
 np.float32(18.758976),
 np.float32(9.227623),
 np.float32(14.880968),
 np.float32(19.254587),
 np.float32(8.607777),
 np.float32(17.4062),
 np.float32(20.047169),
 np.float32(14.710155),
 np.float32(18.222057),
 np.float32(9.418196),
 np.float32(19.69621),
 np.float32(10.966576),
 np.float32(27.154222),
 np.float32(9.223244),
 np.float32(8.879504),
 np.float32(15.616725),
 np.float32(12.684843),
 np.float32(10.73335),
 np.float32(9.479132),
 np.float32(16.28757),
 np.float32(8.445735),
 np.float32(19.

In [ ]:
stack = np.column_stack((range(len(recon_errors)), recon_errors))
score_false = stack[test_labels == 0][:, 1]
score_true = stack[test_labels == 1][:, 1]

plot_histogram_by_class(
    score_false,
    score_true,
    bins=[20, 30],
    model_name=MODEL_NAME,
)

In [ ]:
THRESHOLD_MIN = 0.0
THRESHOLD_MAX = 10

p = figure(
    width=600,
    height=400,
    title=f"{MODEL_NAME}: Threshold Range Exploration",
    x_axis_label="Samples",
    y_axis_label="Reconstruction Error",
)

source = ColumnDataSource(
    dict(index=stack[test_labels == 0][:, 0], error=stack[test_labels == 0][:, 1])
)

p.scatter(
    "index",
    "error",
    fill_alpha=0.6,
    fill_color="crimson",
    line_color=None,
    legend_label="Normal Signals",
    source=source,
)

source = ColumnDataSource(
    dict(index=stack[test_labels == 1][:, 0], error=stack[test_labels == 1][:, 1])
)

p.scatter(
    "index",
    "error",
    fill_alpha=0.6,
    fill_color="indigo",
    line_color=None,
    legend_label="Abnormal Signals",
    source=source,
)

source = ColumnDataSource(
    data=dict(
        index=stack[:, 0],
        threshold_min=np.repeat(THRESHOLD_MIN, stack.shape[0]),
        threshold_max=np.repeat(THRESHOLD_MAX, stack.shape[0]),
    )
)

box = BoxAnnotation(
    bottom=THRESHOLD_MIN,
    top=THRESHOLD_MAX,
    fill_alpha=0.1,
    fill_color="magenta",
    line_color="darkmagenta",
    line_width=1.0,
)
p.add_layout(box)

p.legend.label_text_font_size = "8pt"
p.legend.location = "top_right"
p.title.align = "center"
p.title.text_font_size = "12pt"

p.add_tools(HoverTool(tooltips=[("index", "@index"), ("error", "@error")]))

show(p)

In [ ]:
THRESHOLD_MIN = 0.00
THRESHOLD_MAX = 15
THRESHOLD_STEP = 0.5

thresholds = np.arange(THRESHOLD_MIN, THRESHOLD_MAX + THRESHOLD_STEP, THRESHOLD_STEP)
errors = []

for threshold in thresholds:
    predictions = get_prediction(stack[:, 1], threshold=threshold)
    conf_mat = confusion_matrix(test_labels, predictions)
    errors.append([threshold, conf_mat[1, 0], conf_mat[0, 1]])

errors = np.array(errors)

p = figure(
    width=600,
    height=400,
    title=f"{MODEL_NAME}: Best Threshold Exploration",
    x_axis_label="Reconstruction Error Threshold (%)",
    y_axis_label="# Samples",
)

source = ColumnDataSource(
    data=dict(
        threshold=errors[:, 0], false_negative=errors[:, 1], false_positive=errors[:, 2]
    )
)

p.line(
    x="threshold",
    y="false_negative",
    color="crimson",
    legend_label="False Negative",
    source=source,
)

p.line(
    x="threshold",
    y="false_positive",
    color="indigo",
    legend_label="False Positive",
    source=source,
)

p.legend.label_text_font_size = "8pt"
p.legend.location = "top_left"
p.legend.click_policy = "hide"
p.title.align = "center"
p.title.text_font_size = "12pt"

p.add_tools(
    HoverTool(
        tooltips=[
            ("threshold", "@threshold"),
            ("false_negative", "@false_negative"),
            ("false_positive", "@false_positive"),
        ]
    )
)
show(p)


In [ ]:
THRESHOLD = 10
predictions = get_prediction(stack[:, 1], threshold=THRESHOLD)

plot_confusion_matrix(
    confusion_matrix(test_labels, predictions),
    model_name=MODEL_NAME,
)

print(
    f"Accuracy: {accuracy_score(test_labels, predictions):.2%}, \
Precision: {precision_score(test_labels, predictions):.2%}, \
Recall: {recall_score(test_labels, predictions):.2%}, \
F1: {f1_score(test_labels, predictions):.2%}"
)

Accuracy: 96.91%, Precision: 97.33%, Recall: 96.41%, F1: 96.87%


In [ ]:
plot_roc_curve(
    roc_curve(test_labels, recon_errors),
    roc_auc_score(test_labels, recon_errors),

    model_name=MODEL_NAME
)

In [ ]:
auc=roc_auc_score(test_labels, recon_errors)

print(f"AUC score: {auc:.4f}")

AUC score: 0.9938


In [ ]:
plot_pr_curve(
    precision_recall_curve(test_labels, recon_errors),
    average_precision_score(test_labels, recon_errors),
    model_name=MODEL_NAME
)

In [ ]:
# pAUC score
pauc = compute_partial_auc(test_labels, recon_errors, max_fpr=0.1)
print(f"Unnormalized Partial AUC (FPR ≤ 0.1): {pauc:.4f} or the model performs {(pauc/0.1):.1%} as well as a perfect classifier in the region where FPR ≤ 0.1.")

#pauc/0.1 * 100

Unnormalized Partial AUC (FPR ≤ 0.1): 0.0951 or the model performs 95.1% as well as a perfect classifier in the region where FPR ≤ 0.1.


# New Section

In [ ]:
import tensorflow as tf
from keras.layers import Cropping2D

LATENT_DIM=2

class Sampling(tf.keras.layers.Layer):
  def call(self, inputs):
    """Generates a random sample and combines with the encoder output

    Args:
      inputs -- output tensor from the encoder

    Returns:
      `inputs` tensors combined with a random sample
    """

    # unpack the output of the encoder
    mu, sigma = inputs

    # get the size and dimensions of the batch
    batch = tf.shape(mu)[0]
    dim = tf.shape(mu)[1]

    # generate a random tensor
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))

    # combine the inputs and noise
    return mu + tf.exp(0.5 * sigma) * epsilon

class KLDLayer(tf.keras.layers.Layer):
  def call(self, inputs):
    """Computes the KLD loss and adds it to the model

    Args:
      inputs -- tensor containing (mu, sigma)

    Returns:
      kl_loss -- computed Kullback–Leibler Divergence loss
    """

    # unpack the inputs
    mu, sigma = inputs

    # compute the loss
    kl_loss = 1 + sigma - tf.square(mu) - tf.math.exp(sigma)
    kl_loss = tf.reduce_mean(kl_loss) * -0.5

    # store the result
    self.add_loss(kl_loss)

    return kl_loss

def encoder_layers(inputs, latent_dim):
  """Defines the encoder's layers.
  Args:
    inputs -- batch from the dataset
    latent_dim -- dimensionality of the latent space

  Returns:
    mu -- learned mean
    sigma -- learned standard deviation
    batch_2.shape -- shape of the features before flattening
  """

  # add the Conv2D layers followed by BatchNormalization
  x = tf.keras.layers.Conv2D(filters=40, kernel_size=3, strides=(1,2), padding="same", activation='relu', name="encode_conv1")(inputs)
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Conv2D(filters=40, kernel_size=3, strides=(1,2), padding='same', activation='relu', name="encode_conv2")(x)
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Conv2D(filters=80, kernel_size=3, strides=(1,2), padding='same', activation='relu', name="encode_conv3")(x)

  # assign to a different variable so you can extract the shape later
  batch_2 = tf.keras.layers.BatchNormalization()(x)

  # flatten the features and feed into the Dense network
  x = tf.keras.layers.Flatten(name="encode_flatten")(batch_2)

  # we arbitrarily used 12 units here but feel free to change and see what results you get
  x = tf.keras.layers.Dense(20, activation='relu', name="encode_dense")(x)
  #x = tf.keras.layers.BatchNormalization()(x)

  # add output Dense networks for mu and sigma, units equal to the declared latent_dim.
  mu = tf.keras.layers.Dense(latent_dim, name='latent_mu')(x)
  sigma = tf.keras.layers.Dense(latent_dim, name ='latent_sigma')(x)

  return mu, sigma, batch_2.shape

def encoder_model(latent_dim, input_shape):
  """Defines the encoder model with the Sampling layer
  Args:
    latent_dim -- dimensionality of the latent space
    input_shape -- shape of the dataset batch

  Returns:
    model -- the encoder model
    conv_shape -- shape of the features before flattening
  """

  # declare the inputs tensor with the given shape
  inputs = tf.keras.Input(shape=input_shape, name='encoder_input')

  # get the output of the encoder_layers() function
  mu, sigma, conv_shape = encoder_layers(inputs, latent_dim=LATENT_DIM)

  # feed mu and sigma to the Sampling layer
  z = Sampling()((mu, sigma))

  # feed mu and sigma to the KLD layer
  kl_loss = KLDLayer()((mu, sigma))

  # build the whole encoder model
  model = tf.keras.Model(inputs=inputs, outputs=[z, kl_loss])

  return model, conv_shape

def decoder_layers(inputs, conv_shape):
  """Defines the decoder layers.
  Args:
    inputs -- output of the encoder
    conv_shape -- shape of the features before flattening

  Returns:
    tensor containing the decoded output
  """

  # feed to a Dense network with units computed from the conv_shape dimensions
  units = conv_shape[1] * conv_shape[2] * conv_shape[3]
  x = tf.keras.layers.Dense(units, activation = 'relu', name="decode_dense1")(inputs)
  #x = tf.keras.layers.BatchNormalization()(x)

  # reshape output using the conv_shape dimensions
  x = tf.keras.layers.Reshape((conv_shape[1], conv_shape[2], conv_shape[3]), name="decode_reshape")(x)

  # upsample the features back to the original dimensions
  x = tf.keras.layers.Conv2DTranspose(filters=80, kernel_size=3, strides=(1,2), padding='same', activation='relu', name="decode_conv2d_1")(x)
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Conv2DTranspose(filters=40, kernel_size=3, strides=(1,2), padding='same', activation='relu', name="decode_conv2d_2")(x)
  x = tf.keras.layers.BatchNormalization()(x)

  x = tf.keras.layers.Conv2DTranspose(filters=40, kernel_size=3, strides=(1,2), padding='same', activation='relu', name="decode_conv2d_3")(x)
  x = tf.keras.layers.BatchNormalization()(x)


  return x

def decoder_model(latent_dim, conv_shape):
  """Defines the decoder model.
  Args:
    latent_dim -- dimensionality of the latent space
    conv_shape -- shape of the features before flattening

  Returns:
    model -- the decoder model
  """

  # set the inputs to the shape of the latent space
  inputs = tf.keras.Input(shape=(latent_dim,))

  # get the output of the decoder layers
  outputs = decoder_layers(inputs, conv_shape)

  # declare the inputs and outputs of the model
  model = tf.keras.Model(inputs, outputs)

  return model


def vae_model(encoder, decoder, input_shape):
  """Defines the VAE model
  Args:
    encoder -- the encoder model
    decoder -- the decoder model
    input_shape -- shape of the dataset batch

  Returns:
    the complete VAE model
  """

  # set the inputs
  inputs = tf.keras.Input(shape=input_shape)

  # get z from the encoder output
  z, _ = encoder(inputs)

  # get reconstructed output from the decoder
  reconstructed = decoder(z)

  # define the inputs and outputs of the VAE
  model = tf.keras.Model(inputs=inputs, outputs=reconstructed)

  return model

def get_models(input_shape, latent_dim):
  """Returns the encoder, decoder, and vae models"""
  encoder, conv_shape = encoder_model(latent_dim=latent_dim, input_shape=input_shape)
  decoder = decoder_model(latent_dim=latent_dim, conv_shape=conv_shape)
  vae = vae_model(encoder, decoder, input_shape=input_shape)
  return encoder, decoder, vae

# Get the encoder, decoder and 'master' model (called vae) #input_shape=(256,626,1)
encoder, decoder, vae = get_models(input_shape=(5,80,1), latent_dim=LATENT_DIM)

vae.summary()

# Define the loss functions and optimizers
optimizer = tf.keras.optimizers.Adam()
loss_metric = tf.keras.metrics.Mean()
mse_loss = tf.keras.losses.MeanSquaredError()

Model: "functional_134"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_89 (InputLayer)     │ (None, 5, 80, 1)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_132 (Functional)     │ [(None, 2), ()]        │       124,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ functional_133 (Functional)     │ (None, 5, 80, 40)      │       113,600 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 238,064 (929.94 KB)

 Trainable params: 237,424 (927.44 KB)

 Non-trainable params: 640 (2.50 KB)